In [1]:
!cat './data/id=10_n=5_p=0.5_k=3.cnf'

p cnf 15 41
1 2 3 0
4 5 6 0
7 8 9 0
10 11 12 0
13 14 15 0
-1 -2 0
-1 -3 0
-2 -3 0
-4 -5 0
-4 -6 0
-5 -6 0
-7 -8 0
-7 -9 0
-8 -9 0
-10 -11 0
-10 -12 0
-11 -12 0
-13 -14 0
-13 -15 0
-14 -15 0
-1 -4 0
-2 -5 0
-3 -6 0
-1 -10 0
-2 -11 0
-3 -12 0
-4 -7 0
-5 -8 0
-6 -9 0
-4 -10 0
-5 -11 0
-6 -12 0
-4 -13 0
-5 -14 0
-6 -15 0
-7 -10 0
-8 -11 0
-9 -12 0
-7 -13 0
-8 -14 0
-9 -15 0


In [2]:
import os
import pdb
import random
from collections import namedtuple

import scipy.sparse as sparse
import torch

from cnf import CNF
from util import DataSample, adj, adj_batch, init_edge_attr, to_sparse_tensor


Batch = namedtuple('Batch', ['x', 'adj', 'sol'])
SampleWrapper = namedtuple('SampleWrapper', ['ptype', 'sample'])


def load_dir(path):
    data = []
    for filename in os.listdir(path):
        name, ext = os.path.splitext(filename)
        if ext != '.cnf':
            continue
        f = CNF.from_file(os.path.join(path, filename))
        if name.startswith('uu'):
            continue
        data.append(DataSample(filename, f, adj(f), None))
    return data


def load_dir_wrapper(path):
    data = load_dir(path)
    return [SampleWrapper(ptype, sample) for sample in data]


def init_tensor_wrapper(sample_wrapper, device):
    return (sample_wrapper.ptype, init_tensors(sample_wrapper.sample, device))

In [3]:
import numpy as np

In [4]:
data = load_dir('../data/kcolor/3-5-0.5/')

In [5]:

def adj_sign(n, m, occur):
    i = np.repeat(range(n), [len(lst) for lst in occur])
    j = np.concatenate(occur)
    v = np.ones(len(i), dtype=np.int64)
    return sparse.coo_matrix((v, (i, j)), shape=(n, m))


def adj(f):
    n, m, occur = f.n_variables, len(f.clauses), f.occur_list
    adj_pos = adj_sign(n, m, occur[1 : n + 1])
    adj_neg = adj_sign(n, m, occur[:n:-1])
    return (adj_pos, adj_neg)


def adj_batch(adjs, fstack):
    adjp, adjn = list(zip(*adjs))
    return fstack((sparse.block_diag(adjp), sparse.block_diag(adjn)))


def to_sparse_tensor(x):
    x = x.tocoo()
    i = torch.tensor(np.vstack((x.row, x.col)), dtype=torch.int64)
    v = torch.tensor(x.data, dtype=torch.float32)
    return torch.sparse_coo_tensor(i, v, torch.Size(x.shape))

def init_edge_attr(k):
    return torch.cat(
        (
            torch.tensor([1, 0], dtype=torch.float32).expand(k, 2),
            torch.tensor([0, 1], dtype=torch.float32).expand(k, 2),
        ),
        dim=0,
    )


def normalize(x):
    return 2 * x - 1


def unnormalize(x):
    return (x + 1) / 2


In [6]:
def init_tensors(sample, device):
    # [1,0,0] -> assigned False
    # [0,1,0] -> assigned True
    # [0,0,1] -> clause
    adj = sample.adj
    n, m = adj[0].shape[0], adj[0].shape[1]
    xv = torch.zeros(n, 3, dtype=torch.float32)
    sol = [x if random.random() < 0.5 else -x for x in range(n + 1)]
    xv[torch.arange(n), (torch.tensor(sol[1:]) > 0).long()] = 1
    xv = xv.to(device)
    xc = torch.tensor([0, 0, 1], dtype=torch.float32).repeat(m, 1).to(device)
    xev = init_edge_attr(n).to(device)
    xec = init_edge_attr(m).to(device)
    vadj = to_sparse_tensor(sparse.hstack(adj)).to(device)
    cadj = to_sparse_tensor(sparse.vstack(adj)).t().to(device)
    return Batch((xv, xc, xev, xec), (vadj, cadj), sol)


In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [8]:
tensor = init_tensors(data[53], device)

In [9]:
f = data[3].formula
n,m,occur=f.n_variables, len(f.clauses), f.occur_list
np.repeat(range(n), [len(lst) for lst in occur[:n:-1]])

array([ 0,  0,  0,  0,  0,  1,  1,  1,  1,  1,  2,  2,  2,  2,  2,  3,  3,
        3,  3,  3,  4,  4,  4,  4,  4,  5,  5,  5,  5,  5,  6,  6,  6,  6,
        6,  7,  7,  7,  7,  7,  8,  8,  8,  8,  8,  9,  9,  9,  9, 10, 10,
       10, 10, 11, 11, 11, 11, 12, 12, 12, 13, 13, 13, 14, 14, 14])

In [10]:
np.repeat(range(n), [len(lst) for lst in occur[1:n+1]])

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

In [11]:
"""
def adj_sign(n, m, occur):
    i = np.repeat(range(n), [len(lst) for lst in occur])
    j = np.concatenate(occur)
    v = np.ones(len(i), dtype=np.int64)
    return sparse.coo_matrix((v, (i, j)), shape=(n, m))


def adj(f):
    n, m, occur = f.n_variables, len(f.clauses), f.occur_list
    adj_pos = adj_sign(n, m, occur[1 : n + 1])
    adj_neg = adj_sign(n, m, occur[:n:-1])
    return (adj_pos, adj_neg)
"""

'\ndef adj_sign(n, m, occur):\n    i = np.repeat(range(n), [len(lst) for lst in occur])\n    j = np.concatenate(occur)\n    v = np.ones(len(i), dtype=np.int64)\n    return sparse.coo_matrix((v, (i, j)), shape=(n, m))\n\n\ndef adj(f):\n    n, m, occur = f.n_variables, len(f.clauses), f.occur_list\n    adj_pos = adj_sign(n, m, occur[1 : n + 1])\n    adj_neg = adj_sign(n, m, occur[:n:-1])\n    return (adj_pos, adj_neg)\n'

In [12]:
# to_sparse_tensor_experiment(sparse.coo_array(data[49].adj[0].todense()))

In [13]:
data[3].adj[0].todense()[14]

matrix([[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [14]:
tensor.adj[0].to_dense().view(-1,).shape

torch.Size([870])

In [15]:
tensor.x[2].shape  #edge 1

torch.Size([30, 2])

In [16]:
tensor.adj[0].to_dense().shape

torch.Size([15, 58])

In [17]:
torch.matmul(tensor[0][0], tensor[0][1].transpose(0,1))

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

In [18]:
from vgae import VGAE, GraphEncoder, VGAEEncoder
from train_search import vgae_loss

In [19]:
encoder = VGAE(3,32,16,mlp_arch={'hidden_sizes': [32], 'activation': "ReLU"}, gnn_iter=3, gnn_async=False)

In [20]:
# weak_encoder = VGAEEncoder(3,32,16,mlp_arch={'hidden_sizes': [32], 'activation': "ReLU"}, gnn_iter=3, gnn_async=False)

In [21]:
adj_rec, mean, log_var = encoder(tensor)

In [22]:
vgae_loss(adj_rec, tensor.adj[0], mean, log_var)

tensor(426.3470, grad_fn=<AddBackward0>)

In [23]:
adjref = tensor.adj[0]

In [24]:
_, m = adjref.shape
adjref1, adjref2 = torch.split(adjref.to_dense(), m//2, dim=1)
adj = adjref1 - adjref2

In [31]:
adj_rec

tensor([[8.4397e-02, 2.5994e-08, 9.4079e-01, 6.3401e-01, 5.5460e-02, 1.2074e-01,
         8.1879e-01, 7.0147e-11, 7.2899e-01, 1.0000e+00, 1.0000e+00, 6.0453e-02,
         9.9992e-01, 8.9899e-01, 9.9838e-01, 7.4729e-12, 9.9999e-01, 4.3594e-04,
         1.0000e+00, 9.9642e-01, 4.3379e-11, 4.4682e-06, 2.6908e-08, 9.9228e-01,
         1.0918e-02, 1.5033e-04, 9.9979e-01, 5.0643e-12, 9.5589e-01],
        [1.0904e-05, 4.0763e-09, 8.9375e-01, 1.1441e-02, 1.6655e-04, 2.7593e-06,
         2.2924e-07, 2.0042e-04, 2.0920e-01, 9.9992e-01, 1.0000e+00, 1.2708e-05,
         9.9427e-01, 1.0000e+00, 5.3287e-01, 5.7408e-03, 2.4491e-01, 8.5548e-01,
         1.0000e+00, 9.9920e-01, 2.5079e-04, 1.9758e-11, 1.4222e-01, 1.0000e+00,
         5.6219e-06, 3.6897e-01, 1.7739e-01, 4.2417e-04, 5.4920e-02],
        [5.7469e-06, 9.0720e-05, 2.5114e-07, 9.9977e-01, 1.0000e+00, 1.0410e-04,
         4.0634e-11, 2.2994e-03, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.1331e-06,
         9.7567e-01, 4.4257e-10, 9.7903e-01, 9.149

In [26]:
(adjref1.numpy() + adjref2.numpy())

array([[1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0

In [27]:
vgae = torch.load('../results/vgae_mixed_all_v1.pth')

In [28]:
adjtest, _, _ = vgae(tensor)

In [29]:
adjtest.round()

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

In [30]:
adjtest

tensor([[1.5735e-04, 1.3173e-07, 6.9354e-07, 4.9724e-06, 6.2692e-06, 3.8925e-06,
         2.2955e-06, 1.4122e-04, 1.4366e-05, 4.2173e-06, 3.8106e-07, 9.1548e-05,
         3.4579e-05, 6.1913e-05, 6.7267e-05, 5.3562e-06, 1.1032e-05, 1.9796e-05,
         1.8918e-05, 1.1570e-06, 2.7022e-08, 2.2878e-06, 8.9005e-08, 2.6242e-06,
         1.3067e-06, 8.1779e-05, 9.8472e-07, 2.4191e-06, 8.6312e-07],
        [1.6027e-04, 3.6427e-06, 2.2084e-08, 7.9706e-06, 2.3123e-08, 6.8429e-05,
         5.8749e-06, 1.1878e-07, 3.2243e-06, 3.3107e-08, 1.0654e-08, 5.7488e-06,
         5.2984e-07, 1.2576e-06, 2.2899e-07, 4.5911e-05, 4.9331e-08, 2.8434e-07,
         1.7513e-07, 4.6050e-06, 1.0990e-06, 6.5403e-05, 1.5602e-07, 1.0984e-05,
         1.0260e-05, 4.2227e-06, 2.7737e-05, 3.3318e-06, 7.3184e-05],
        [2.9061e-06, 6.1817e-06, 5.2764e-06, 2.7553e-07, 6.9735e-07, 5.7716e-06,
         2.9193e-06, 1.0713e-05, 1.8243e-05, 2.7116e-06, 1.9580e-07, 7.5119e-06,
         3.3990e-07, 1.0685e-06, 2.2716e-06, 1.197

In [50]:

def test_vgae_model_gradients(model, tensor):
    # Create a random input adjacency matrix
    adjref = tensor.adj[0]

    # Create an instance of your VGAE model  # Replace with your actual model class

    # Set the model to evaluation mode
    model.eval()

    # Enable gradient computation for the input adjacency matrix
    # tensor.adj.requires_grad = True
    tensor.x[0].requires_grad = True
    tensor.x[1].requires_grad = True
    tensor.x[2].requires_grad = True
    tensor.x[3].requires_grad = True

    # Forward pass through the VGAE model
    adj_reconstructed, mean, log_var = model(tensor)

    # Compute the loss using your vgae_loss function
    loss = vgae_loss(adj_reconstructed, adjref, mean, log_var)

    # Perform gradient checking
    test_passed = torch.autograd.gradcheck(lambda adj: vgae_loss(*model(adj)), (tensor,), eps=1e-6, atol=1e-4)

    if test_passed:
        print("Gradient check passed!")
    else:
        print("Gradient check failed.")


In [51]:
test_vgae_model_gradients(encoder, tensor)

ValueError: gradcheck expects at least one input tensor to require gradient, but none of the them have requires_grad=True.

In [44]:
tensor.x

(tensor([[0., 1., 0.],
         [0., 1., 0.],
         [1., 0., 0.],
         [1., 0., 0.],
         [0., 1., 0.],
         [1., 0., 0.],
         [1., 0., 0.],
         [1., 0., 0.],
         [1., 0., 0.],
         [0., 1., 0.],
         [1., 0., 0.],
         [0., 1., 0.],
         [1., 0., 0.],
         [0., 1., 0.],
         [1., 0., 0.]]),
 tensor([[0., 0., 1.],
         [0., 0., 1.],
         [0., 0., 1.],
         [0., 0., 1.],
         [0., 0., 1.],
         [0., 0., 1.],
         [0., 0., 1.],
         [0., 0., 1.],
         [0., 0., 1.],
         [0., 0., 1.],
         [0., 0., 1.],
         [0., 0., 1.],
         [0., 0., 1.],
         [0., 0., 1.],
         [0., 0., 1.],
         [0., 0., 1.],
         [0., 0., 1.],
         [0., 0., 1.],
         [0., 0., 1.],
         [0., 0., 1.],
         [0., 0., 1.],
         [0., 0., 1.],
         [0., 0., 1.],
         [0., 0., 1.],
         [0., 0., 1.],
         [0., 0., 1.],
         [0., 0., 1.],
         [0., 0., 1.],
         